In [1]:
import warnings
warnings.filterwarnings('ignore')

from astropy.table import Table, hstack, vstack, join, MaskedColumn
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as ac
import astropy.units as au
from glob import glob
from spectral_cube import SpectralCube
import scipy 
from reproject import reproject_interp
from scipy.optimize import curve_fit

from tqdm.auto import tqdm 
from astropy.io import fits
import matplotlib as mpl
import pyregion
import aplpy
import math
import os
import pickle

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in" 

from analysis_phangs_hst import dendro_misc, dendro_pressures, dendro_sb99, dendro_props, dendro_pressures

In [2]:
"""Load properties of the hst nebula catalouge"""
props_all_file = '../../analysis/dendro/props_all/02_props_all_3sig.fits'
props_all = Table.read(props_all_file)

In [3]:
SB99models = dendro_sb99.get_SB99models()
SB99models_arr = dendro_sb99.get_SB99models_arr()

In [4]:
#### Defines
te = props_all['T_N2_REFIT']*au.K
te[np.isnan(te)] = 8000*au.K
ne = props_all['ne']

# radius = props_all['mean_sigma_pc']
# lha = calculate_luminosity(props_all['HA6562_FLUX_CORR']*1e-20, props_all['dist_pc'])

radius = props_all['radius_trunkclose_pc']
lha = props_all['ha_lum_hst']

In [5]:
# Get Lfrac(t) - luminosity fraction for given age (from clusters)
props_all['lbol/lha'] = dendro_props.get_lfrac(props_all, SB99models, showplot=False)

# Get bolometric luminosity from factor - either constant or vary with age
props_all['lbol'] = lbol = dendro_props.get_lbol(lha.quantity, conv=props_all['lbol/lha'].quantity.value)
props_all['lbol_const'] = lbol_const = dendro_props.get_lbol(lha.quantity, conv=138) 

# Get electron density from radius
props_all['ne_radlim'] = ne_radlim = dendro_props.get_ne(lha, radius, te)

Calculate Lfrac:   0%|          | 0/2878 [00:00<?, ?it/s]

In [6]:
# Get PDIR
pdir = dendro_pressures.get_pdir(lbol, radius)
pdir_const = dendro_pressures.get_pdir(lbol_const, radius)

props_all['pdir'] = np.log10(pdir.value) * pdir.unit
props_all['pdir_const'] = np.log10(pdir_const.value) * pdir.unit

In [7]:
### Get PTHERM
pth = dendro_pressures.get_pth(ne.quantity, te)
pth_radlim = dendro_pressures.get_pth(ne_radlim, te)

props_all['pth'] = np.log10(pth.value) * pth.unit
props_all['pth_radlim'] = np.log10(pth_radlim.value) * pth.unit

# mask column 
props_all['pth'] = MaskedColumn(props_all['pth'], mask=np.isnan(props_all['pth']))

In [8]:
# Get Lmech, Mdot, vwind
lmec, mdot, windvelo = dendro_sb99.get_sb99props(SB99models_arr, props_all, showplots=False)

Calculate Lmech, Mdot, Vwind:   0%|          | 0/2878 [00:00<?, ?it/s]

In [9]:
# ### Get PWIND
pwind = dendro_pressures.get_pwind(mdot, windvelo, radius)
props_all['pwind'] = np.log10(pwind.value) * pwind.unit

In [10]:
dendro_misc.save_pickle(props_all, props_all_file.replace('02','03').replace('fits','pickel')) 
dendro_pressures.save_fits(props_all, props_all_file)

[INFO] [save_pickle] Saved to ../../analysis/dendro/props_all/03_props_all_3sig.pickel
[INFO] [save_fits] Saved to ../../analysis/dendro/props_all/03_props_all_3sig.fits
